# Spotify Track Analytics Popularity Prediction: ML Models

### Introduction

This notebook extends the Spotify track exploration work by building end-to-end ML workflows that predict a song's popularity score from its audio features and metadata. We will establish a clean dataset, prepare modeling pipelines, and compare baseline algorithms to understand which inputs drive popularity predictions.

---

For establishing furhter ML modeling, it is necessary to import libraries

In [ ]:
import pandas as pd                 #import pandas for data manipulation
import numpy as np             #import numpy for numerical operations
import seaborn as sns       #import seaborn for data visualization
import matplotlib.pyplot as plt                 #import matplotlib for plotting
from sklearn.model_selection import train_test_split  #import train_test_split for splitting data                                                      
from sklearn.preprocessing import StandardScaler, OneHotEncoder #import preprocessing tools
from sklearn.compose import ColumnTransformer #import ColumnTransformer for preprocessing
from sklearn.pipeline import Pipeline                  # import Pipeline for creating ML pipelines
from sklearn.linear_model import LinearRegression   #import Linear Regression model
from sklearn.ensemble import RandomForestRegressor  #import Random Forest Regressor
from xgboost import XGBRegressor  #import XGBoost Regressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  #import evaluation metrics
from sklearn.preprocessing import StandardScaler  #import StandardScaler for feature scaling
from sklearn.decomposition import PCA  #import PCA for dimensionality reduction
from sklearn.cluster import KMeans   #import KMeans for clustering


As the first step the cleaned in [previous notebook](https://github.com/YShutko/CI_spotify_track_analysis/blob/2972bbfa3227c3ab665618cfcf3cae74f5215dbe/notebooks/Spotify_track_analysis.ipynb) dataset.

In [ ]:
# Load dataset
df = pd.read_csv('../data/spotify_cleaned_data.csv')  # Adjust path as needed
df.head ()

---

### 1. Linear Regression

#### Introduction

In this part of the project, we build a Linear Regression model to predict a track’s popularity based on its audio features (danceability, energy, loudness, valence, etc.) and engineered features (mood, intensity). Linear Regression helps us understand how each feature influences popularity and serves as a simple baseline model. Before training, we preprocess the data using scaling and one-hot encoding to ensure all features are on comparable scales and properly formatted for the model. This baseline evaluation provides a reference point for comparing more advanced machine learning models later in the project.

#### Model building

In the following steps feature engeneering is performed. 

Linear regression can only learn straight-line relationships.
But many musical characteristics are nonlinear and interact with each other.
By multiplying two features, we allow the model to learn interaction patterns that linear regression cannot capture naturally.

Energy * loudness helps the model understand: "How intense, powerful, and high-impact the song feels."
This is strongly related to genre and mood, which correlate with popularity and listener engagement.
Without this feature:
The model only sees energy and loudness independently
→ losing important relationships
→ decreasing predictive power

"Mood" is not directly in the dataset, but it is one of the strongest predictors of human music preference.
These 3 features measure:
* Danceability → how rhythmically engaging
* Energy → how active or intense
* Valence → positivity/happiness
Averaging them gives a general emotional / vibe score.
What it gives the model:
* A single, compact representation of “how the song feels”
* Helps cluster songs by vibe
* Helps regression capture patterns like:
* happy energetic songs tend to be more popular
* sad low-energy songs tend to rank differently

Why linear regression benefits:
This creates a latent feature (a hidden variable) that captures something meaningful which raw variables alone cannot represent.

Spotify dataset stores "explicit" as 0/1 integers, which machine learning treats as numeric.
But it is not numeric — it’s categorical (yes/no).

If not converted:
* The model will treat explicit = 1 as “higher” than explicit = 0
* It will try to fit a line like: popularity = ... + 3.7 * explicit
* This is incorrect, because explicitness is not a quantity

After converting to Boolean:
* True / False becomes a category
* It will be one-hot encoded correctly (explicit_True, explicit_False)

This avoids incorrect numerical influence and improves model interpretation.

In [ ]:
# Interaction: energy * loudness (how intense & loud a track feels)
df["energy_loudness"] = df["energy"] * df["loudness"]

# Simple "mood" score: combination of danceability, energy, valence
df["mood"] = (df["danceability"] + df["energy"] + df["valence"]) / 3

#explicit is not boolean already, convert it
df["explicit"] = df["explicit"].astype(bool)

Next step is to select the prediction target and features.
`popularity` is used as the target variable `y`. `X` is builded from sound characteristics (tempo, danceability, energy, etc.), boolean/context flags (explicit, key, mode), and any engineered aggregates. 

In [ ]:
target = "popularity"

numeric_features = [
    "danceability",
    "energy",
    "key",
    "loudness",
    "mode",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "time_signature",
    # engineered:
    "duration_min",
    "energy_loudness",
    "mood",
]

categorical_features = [
    "explicit",      # True/False
    "track_genre",   # 125 genres, will be one-hot encoded
]

X = df[numeric_features + categorical_features]
y = df[target]

Preprocessing: Scaling + One-Hot Encoding

StandardScaler
Scales all numeric features so they have mean = 0 and standard deviation = 1.
This prevents large-valued columns (e.g., duration_ms) from dominating the model.

OneHotEncoder
Converts categorical features (e.g., track_genre, explicit) into numeric vectors.
handle_unknown="ignore" prevents errors if unseen categories appear in test data.

ColumnTransformer
Applies scaling to numeric columns and one-hot encoding to categorical columns in a single unified preprocessing step.
Ensures consistent, leak-free preprocessing inside the ML pipeline.

In [ ]:
# Preprocessing: scaling + one-hot encoding
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Pipeline: Preprocessing → Linear Regression

The Pipeline combines all steps into one model:

First, it applies preprocessing
(scaling numeric features + one-hot encoding categorical features)

Then, it runs LinearRegression on the transformed data.

This ensures the same preprocessing is applied during both training and prediction, preventing data leakage and keeping the workflow clean and reproducible.

In [ ]:

# Build pipeline: preprocessing → linear regression
lr= Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("regressor", LinearRegression())
    ]
)


Train/Test Split
train_test_split divides the dataset into two parts:
* Training set (80%) → used to fit the model
* Test set (20%) → used to evaluate how well the model generalizes

random_state=42 ensures the split is reproducible every time you run the code.

In [ ]:

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


fit the Linear regression model

In [ ]:
# Fit model
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)


And save model

In [ ]:
# os.makedirs('../model/', exist_ok=True)  # Create directory if it doesn't exist
# joblib.dump(lr, '../model/spotify_lr.pkl') # Save the model

Model Evaluation
After predicting values on the test set (y_pred), we calculate key regression metrics:
* MAE (Mean Absolute Error): Average absolute difference between predicted and actual popularity.
* RMSE (Root Mean Squared Error): Penalizes large errors more strongly; shows overall prediction error.
* R² Score: Measures how much variance in popularity the model explains (1 = perfect fit, 0 = no predictive power).

These metrics tell us how well the linear regression model performs on unseen data.

In [ ]:
# Evaluate
y_pred = lr.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Linear Regression results:")
print(f"MAE  : {mae:.3f}")
print(f"RMSE : {rmse:.3f}")
print(f"R^2  : {r2:.3f}")

* R² = 0.25 means:Weak model fit
Linear relationships between features and target are limited
Model captures only basic trends (e.g., louder = slightly more popular)

* MAE ≈ 14 → Not acceptable for predictive tasks:
A good model should have MAE < 8 for Spotify popularity
Industry models often get MAE ≈ 6–10 by adding external metadata

* RMSE ≈ 19 → High variance in the errors:
The model makes some very large mistakes
Linear Regression underfits the data
Popularity is nonlinear + influenced by many missing factors

Numeric feature importance

In [ ]:
# Extract feature names from the preprocessing step
feature_names = lr.named_steps["preprocess"].get_feature_names_out()

# Extract coefficients from the linear regression model
coefficients = lr.named_steps["regressor"].coef_

# Build importance dataframe
importance_df = pd.DataFrame({
    "feature": feature_names,
    "coefficient": coefficients,
    "abs_importance": np.abs(coefficients)
}).sort_values("abs_importance", ascending=False)

numeric_importance = importance_df[
    ~importance_df["feature"].str.contains("track_genre")
]

numeric_importance.head(30)



Top Predictors:
* explicit (True/False) → explicit content has the strongest effect among numeric/categorical non-genre features.
    * Negative/positive signs cancel because both dummy columns appear; importance is the magnitude.
* danceability → more danceable tracks tend to be more popular.
* valence → happier, more positive-sounding songs tend to be more popular.
* acousticness (negative) → high acousticness lowers predicted popularity.
* mode and energy → moderately contribute; higher energy slightly increases popularity.
* speechiness (negative) → tracks with more spoken content (rap, spoken word) tend to score lower.
* mood → higher combined mood (danceability + energy + valence) improves popularity.
* energy_loudness → intensity contributes but less than basic features.

Lower-Influence Features
* instrumentalness → instrumental tracks slightly decrease popularity.
* time_signature, tempo, loudness, liveness → very small effect.
* key, duration_min → minimal influence.

In [ ]:
top_num = numeric_importance.head(20)

plt.figure(figsize=(12, 6))
plt.bar(top_num["feature"], top_num["abs_importance"])
plt.xticks(rotation=90)
plt.title("Top 20 Numeric Feature Importances")
plt.tight_layout()
plt.show()

Interpretation:
* Popularity is influenced more by overall vibe (danceability, valence, energy) than by technical music properties (tempo, key, duration).
* Explicit content and genre dominate, but once genres are removed, the model relies mainly on danceability, positivity, and acousticness.
* Many audio features contribute only weakly, explaining why the linear regression model achieves low R².

Top 20 genres from model

In [ ]:
genre_importance = importance_df[
    importance_df["feature"].str.contains("track_genre")
]

genre_importance.head(20)

Genres with Strong Negative Impact on Popularity.

These genres have large negative coefficients, meaning being in this genre reduces predicted popularity:
* iranian (strongest negative)
* romance
* pop film
* latin
* detroit techno
* chicago house
* jazz
* chill
* kids

These may have smaller listener bases or niche audience groups in the dataset, leading to lower average popularity.

Genres with large positive coefficients contribute to higher popularity predictions:
* k-pop (largest positive)
* sad
* indian
* country
* grunge
* sertanejo
* anime

These styles have broad, dedicated fanbases or strong cultural streaming support.

In [ ]:
top_genres = genre_importance.head(20)

plt.figure(figsize=(12, 6))
plt.bar(top_genres["feature"], top_genres["abs_importance"])
plt.xticks(rotation=90)
plt.title("Top 20 Genres Importances")
plt.tight_layout()
plt.show()

Key insights:
* Genre is more important than audio features for predicting popularity in this dataset.
* The model is essentially learning each genre’s average popularity level.
* This is common in Linear Regression, because genre directly encodes who listens to the music, not just how it sounds.

Why Linear Regression behaves this way
* One-hot encoded genres act like labels for entire listener communities.
* Popularity varies drastically across communities (e.g., k-pop vs. techno).
* These differences produce much larger coefficients than scaled numeric features.

#### Conclusion

The linear regression baseline offers a transparent look at how each standardized feature impacts predicted popularity, making it ideal for initial diagnostics. However, any systematic under/over-prediction patterns or mediocre MAE/R^2 results highlight the need for richer models (tree ensembles, gradient boosting) and potentially more expressive feature engineering.

---

#### What to Do Next to Improve the Model
1. Switch from Linear Regression → Tree-based models
They outperform linear models for this dataset:
* RandomForestRegressor
* XGBoost
* GradientBoostingRegressor
* LightGBM

They capture nonlinear relationships.

2. Add powerful derived features
* Track age (release_year)
* Artist popularity
* Genre grouped into macro-genres
* Energy × Valence (mood)
* Loudness × Danceability

3. Remove low-variance or useless features (key, mode, time_signature barely matter)

4. Try a classification approach
Predict popularity classes instead of exact numbers:
* 0–40 = low
* 41–70 = medium
* 71–100 = high
Classification performs better than regression on music data.

#### 2. Tree based models (RandomForest, XGBoost)

##### 2.1. Additional feature engineering and preprocessing

As it was mentioned above, for better models performance it is necessary to perform additional feature engineering.

In [ ]:
# 2.1 Artist popularity (mean popularity of all tracks by that artist)
df["artist_popularity"] = df.groupby("artists")["popularity"].transform("mean")


In [ ]:
# 2.2 Macro-genre from track_genre
def map_macro_genre(g):
    g = str(g).lower()
    if "pop" in g:
        return "Pop"
    elif "rock" in g:
        return "Rock"
    elif "hip hop" in g or "rap" in g or "trap" in g:
        return "Hip-Hop/Rap"
    elif "r&b" in g or "soul" in g:
        return "R&B/Soul"
    elif "electro" in g or "techno" in g or "house" in g or "edm" in g or "dance" in g:
        return "Electronic/Dance"
    elif "metal" in g or "hardcore" in g:
        return "Metal/Hardcore"
    elif "jazz" in g or "blues" in g:
        return "Jazz/Blues"
    elif "classical" in g or "orchestra" in g or "piano" in g:
        return "Classical"
    elif "latin" in g or "reggaeton" in g or "sertanejo" in g or "samba" in g:
        return "Latin"
    elif "country" in g:
        return "Country"
    elif "folk" in g or "singer-songwriter" in g:
        return "Folk"
    elif "indie" in g or "alternative" in g:
        return "Indie/Alternative"
    else:
        return "Other"

df["macro_genre"] = df["track_genre"].apply(map_macro_genre)

In [ ]:
# 2.4 Interaction features
df["energy_valence"] = df["energy"] * df["valence"]              # mood-like interaction
df["loudness_danceability"] = df["loudness"] * df["danceability"] # intense + danceable

Select features (drop key, mode, time_signature)

In [ ]:
# And select our features for future models


target = "popularity"

numeric_features = [
    "danceability",
    "energy",
    "loudness",
    "speechiness",
    "acousticness",
    "instrumentalness",
    "liveness",
    "valence",
    "tempo",
    "duration_min",
    "artist_popularity",
    "energy_valence",
    "loudness_danceability",
]

# key, mode, time_signature are intentionally NOT included (removed as low-value)

categorical_features = ["explicit", "macro_genre"]

# Prepare data
X = df[numeric_features + categorical_features]
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Preprocessor (no scaling for trees, just one-hot)

In [ ]:

# Preprocessing: scaling + one-hot encoding
numeric_transformer = "passthrough"  # trees don't need scaling
categorical_transformer = OneHotEncoder(handle_unknown="ignore") # one-hot encode categorical features


preprocessor = ColumnTransformer(               
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

##### 2.2 Random Forrest Model

In [ ]:
# 2.2 Random Forrest Model

rf_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("regressor", RandomForestRegressor(
            n_estimators=300,
            max_depth=None,
            random_state=42,
            n_jobs=-1
        )),
    ]
)
# Fit model
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)



In [ ]:
#joblib.dump(rf_model, '../model/spotify_rf.pkl') 

In [ ]:
# Evaluate Random Forest
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)

print("Random Forest Results")
print(f"MAE  : {rf_mae:.3f}")
print(f"RMSE : {rf_rmse:.3f}")
print(f"R^2  : {rf_r2:.3f}")


In [ ]:
rf_features = rf_model.named_steps["preprocess"].get_feature_names_out()
rf_importances = rf_model.named_steps["regressor"].feature_importances_

rf_importance_df = pd.DataFrame({
    "feature": rf_features,
    "importance": rf_importances
}).sort_values("importance", ascending=False)

rf_importance_df.head(20)


In [ ]:
plt.figure(figsize=(12, 6))
top_rf = rf_importance_df.head(20)
plt.bar(top_rf["feature"], top_rf["importance"])
plt.xticks(rotation=90)
plt.title("Random Forest Feature Importance (Top 20)")
plt.tight_layout()
plt.show()


### Conclusion

Random Forest significantly improves predictive performance compared to Linear Regression.  
It captures **nonlinear relationships** and **feature interactions** between audio characteristics and popularity.  

Feature importance becomes more interpretable: macro-genres, mood-related interactions, loudness, danceability, and acousticness have strong influence.  
Random Forest is robust, less sensitive to outliers, and provides a strong benchmark for more advanced models like XGBoost.


In [ ]:
# XGBoost Regressor


xgb_model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("regressor", XGBRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            objective="reg:squarederror"
        )),
    ]
)
# Fit model
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# Evaluate XGBoost
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_r2 = r2_score(y_test, xgb_pred)

print("XGBoost Results")
print(f"MAE  : {xgb_mae:.3f}")
print(f"RMSE : {xgb_rmse:.3f}")
print(f"R^2  : {xgb_r2:.3f}")


save model

In [ ]:
#joblib.dump(xgb_model, '../model/spotify_xgb.pkl') 

In [ ]:
# Extract feature importances
xgb_features = xgb_model.named_steps["preprocess"].get_feature_names_out()
xgb_importances = xgb_model.named_steps["regressor"].feature_importances_

# Build importance dataframe
xgb_importance_df = pd.DataFrame({
    "feature": xgb_features,
    "importance": xgb_importances
}).sort_values("importance", ascending=False)

xgb_importance_df.head(20)


In [ ]:
# Plot XGBoost feature importances
plt.figure(figsize=(12, 6))
top_xgb = xgb_importance_df.head(20)
plt.bar(top_xgb["feature"], top_xgb["importance"])
plt.xticks(rotation=90)
plt.title("XGBoost Feature Importance (Top 20)")
plt.tight_layout()
plt.show()


### Conclusion

XGBoost delivers the best performance among all tested models.  
It uses gradient boosting to iteratively improve predictions and can model complex, nonlinear relationships in the data.  

XGBoost handles the engineered features particularly well, boosting performance on track-age, artist popularity, macro-genres, and mood-based interactions.  
This model provides the highest accuracy and best generalization, making it the preferred approach for popularity prediction.
